In [1]:
# 스파크 세션 생성
import pyspark
from pyspark.sql import SparkSession

# SynapseML : 마이크로소프트에서 만든 스파크용 ML 라이브러리
# https://microsoft.github.io/SynapseML/
# config에서 synapseml 모듈 불러오기
spark = SparkSession.builder \
    .appName("Spark study - 240102") \
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.2") \
    .getOrCreate()

131072x1 화면 크기가 잘못됐습니다. 문제가 예상됩니다
24/01/03 09:51:54 WARN Utils: Your hostname, KJH-DESKTOP resolves to a loopback address: 127.0.1.1; using 192.168.69.220 instead (on interface eth0)
24/01/03 09:51:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/kjh/app/spark/3.5.0/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kjh/.ivy2/cache
The jars for the packages stored in: /home/kjh/.ivy2/jars
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e669f35f-d211-420f-938b-83e37eaa4641;1.0
	confs: [default]
	found com.microsoft.azure#synapseml_2.12;1.0.2 in central
	found com.microsoft.azure#synapseml-core_2.12;1.0.2 in central
	found org.apache.spark#spark-avro_2.12;3.4.1 in central
	found org.tukaani#xz;1.9 in central
	found commons-lang#commons-lang;2.6 in central
	found org.scalactic#scalactic_2.12;3.2.14 in central
	found org.scala-lang#scala-reflect;2.12.15 in central
	found io.spray#spray-json_2.12;1.3.5 in central
	found com.jcraft#jsch;0.1.54 in central
	found org.apache.httpcomponents.client5#httpclient5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.1.3 in central
	found org.slf4j#slf4j-api;1.7.25 in centra

In [2]:
df_albums = spark.read.parquet("file:///home/kjh/data/Spotify/albums/*.parquet")
df_artists = spark.read.parquet("file:///home/kjh/data/Spotify/artists/*.parquet")

In [3]:
df_albums.show(5)
df_artists.show(5)

24/01/03 09:52:06 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU



+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+------------+-----------+-----------+--------+------------+------------+
|            album_id|          album_name|           artist_id| album_type|               label|     album_image_url|            track_id|          track_name|track_number|disc_number|duration_ms|explicit|release_date|total_tracks|
+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+------------+-----------+-----------+--------+------------+------------+
|6Bi5Q5VIzGz0c6zat...|Victoria's Secret...|0LyfQWJT6nXafLPZq...|compilation|Warner Music Grou...|https://i.scdn.co...|4FU8T0t7YASGlMaA8...|   Victoria's Secret|           1|          1|     176124|    true|  2022-09-16|          50|
|6Bi5Q5VIzGz0c6zat...|Victoria's Secret...|0LyfQWJT6nXafLPZq...|comp

24/01/03 09:52:07 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU



In [4]:
df_albums.createOrReplaceTempView('albums')
df_artists.createOrReplaceTempView('artists')

In [5]:
df = spark.sql("""
               SELECT ar.*, al.*
               FROM albums al
               INNER JOIN artists ar
               ON al.artist_id = ar.artist_id
               ORDER BY ar.artist_name
               """)

In [6]:
df.show(5)

+--------------------+------------+--------------------+--------------------+---------+--------------------+----------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------+-----------+-----------+--------+------------+------------+
|           artist_id| artist_name|               genre|    artist_image_url|followers|            album_id|      album_name|           artist_id|album_type|               label|     album_image_url|            track_id|          track_name|track_number|disc_number|duration_ms|explicit|release_date|total_tracks|
+--------------------+------------+--------------------+--------------------+---------+--------------------+----------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------+-----------+-----------+--------+------------+------------+
|0h0C8mMT21p0Ixq6q...|$tupid Young|            cali rap|ht

In [7]:
df = df.drop('artist_id', 'artist_image_url', 'album_id', 'album_image_url', 'track_id')

In [8]:
df.show(5)

+------------+--------------------+---------+----------------+----------+------------------+--------------------+------------+-----------+-----------+--------+------------+------------+
| artist_name|               genre|followers|      album_name|album_type|             label|          track_name|track_number|disc_number|duration_ms|explicit|release_date|total_tracks|
+------------+--------------------+---------+----------------+----------+------------------+--------------------+------------+-----------+-----------+--------+------------+------------+
|$tupid Young|asian american hi...|   131870|From Here On Out|     album|Afficials / EMPIRE|10 Toes Down (fea...|           1|          1|     158027|    true|  2020-12-18|          12|
|$tupid Young|            cali rap|   131870|From Here On Out|     album|Afficials / EMPIRE|10 Toes Down (fea...|           1|          1|     158027|    true|  2020-12-18|          12|
|$tupid Young|asian american hi...|   131870|From Here On Out|     alb

In [9]:
df.count()

547978